<h1>PREDIÇÃO DO VALOR DO FRETE INTERNACIONAL EM UMA OPERAÇÃO DE IMPORTAÇÃO</h1>
<hr>

## Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from sklearn.metrics import median_absolute_error,r2_score

import re

## Coleta, Processamento e Tratamento dos dados
---


### Dados das Importações Brasileiras
Link: [Ministério da Economia](https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip)

In [ ]:
# Coletando os dados das importações Brasileiras e carregando no dataset df_importacao (Tópico 2 do TCC)
df_importacao = pd.read_csv("/content/drive/MyDrive/IMP_COMPLETA.csv",sep=';')

In [ ]:
# Informações do dataset df_importacao (Tópico 2.1 do TCC)
df_importacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36389808 entries, 0 to 36389807
Data columns (total 13 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   CO_NCM      int64 
 3   CO_UNID     int64 
 4   CO_PAIS     int64 
 5   SG_UF_NCM   object
 6   CO_VIA      int64 
 7   CO_URF      int64 
 8   QT_ESTAT    int64 
 9   KG_LIQUIDO  int64 
 10  VL_FOB      int64 
 11  VL_FRETE    int64 
 12  VL_SEGURO   int64 
dtypes: int64(12), object(1)
memory usage: 3.5+ GB


In [ ]:
# Filtrando dados dos últimos 10 anos (2012 - 2022) (Tópico 2.1 do TCC)
df_importacao = df_importacao[df_importacao.CO_ANO >= 2012]

# Filtrando as 20 NCM mais importadas em valor FOB (Topico 2.1 do TCC)
df = df_importacao.groupby(['CO_NCM']).sum().sort_values(by=['VL_FOB'],ascending=False)
fob_total = df.VL_FOB.sum()
df = df.iloc[:20,:][['VL_FOB']]
df.reset_index(inplace=True)
df_importacao = df_importacao[df_importacao.CO_NCM.isin(df.CO_NCM)]

In [ ]:
# Percentual que essas NCM representam no valor FOB (Tópico 2.1 do TCC)
maiores_ncm = df.iloc[:20,:]
fob_maiores_ncm = maiores_ncm.VL_FOB.sum()
perc_ncm = fob_maiores_ncm/fob_total
print(f'O percentual que essas NCM representam no valor FOB é de {perc_ncm:.1%}')

O percentual que essas NCM representam no valor FOB é de 25.5%


In [ ]:
# Informações do dataset df_importacao após filtragem do escopo (Tópico 2.1 do TCC)
df_importacao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117814 entries, 17700319 to 36389722
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   CO_ANO      117814 non-null  int64 
 1   CO_MES      117814 non-null  int64 
 2   CO_NCM      117814 non-null  int64 
 3   CO_UNID     117814 non-null  int64 
 4   CO_PAIS     117814 non-null  int64 
 5   SG_UF_NCM   117814 non-null  object
 6   CO_VIA      117814 non-null  int64 
 7   CO_URF      117814 non-null  int64 
 8   QT_ESTAT    117814 non-null  int64 
 9   KG_LIQUIDO  117814 non-null  int64 
 10  VL_FOB      117814 non-null  int64 
 11  VL_FRETE    117814 non-null  int64 
 12  VL_SEGURO   117814 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 12.6+ MB


In [ ]:
# Visualização do cabeçalho do dataset df_importacao (Tópico 2.1 do TCC)
df_importacao.head()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
17700319,2012,6,85299020,10,351,SP,4,817700,35,35,3252,538,3
17700372,2012,2,31042090,10,149,GO,1,817800,6900000,6900000,3656925,207000,6769
17700388,2012,8,85299020,10,23,SP,1,817800,0,0,167,0,0
17700566,2012,12,84119100,10,149,RJ,4,817700,511,511,1280002,5618,0
17700804,2012,11,84119100,10,249,MG,4,817700,200,200,568191,2602,0


### Tabelas auxiliares
---
Link: [Tabela Códigos](https://balanca.economia.gov.br/balanca/bd/tabelas/TABELAS_AUXILIARES.xlsx)

Indice Excel usado
* 1 (NO_NCM_POR): Descrição NCM
* 10 (CO_PAIS): País de origem do produto
* 14 (CO_VIA): Meio de transporte da mercadoria
* 15 (CO_URF): Local de chegada no Brasil da mercadoria


In [ ]:
# Coletando os dados auxiliares das importações Brasileiras e carregando nos datasets df_ncm, df_pais, df_via e df_urf (Tópico 2 do TCC)


#url = "https://github.com/Migliorin/Comex_Data_Analysis/blob/main/datasets/TABELAS_AUXILIARES.xlsx?raw=true"
#xl_file = pd.ExcelFile(url)

xl_file = pd.ExcelFile('/content/TABELAS_AUXILIARES.xlsx')
df_ncm = xl_file.parse('1')
df_pais = xl_file.parse('10')
df_via = xl_file.parse('14')
df_urf = xl_file.parse('15')

In [ ]:
# Informações do dataset df_ncm (Tópico 2.2.1 do TCC)
df_ncm.info()

In [ ]:
# Informações do dataset df_pais (Tópico 2.2.2 do TCC)
df_pais.info()

In [ ]:
# Informações do dataset df_via (Tópico 2.2.3 do TCC)
df_via.info()

In [ ]:
# Informações do dataset df_urf (Tópico 2.2.4 do TCC)
df_urf.info()

### Substituição código
---


In [ ]:
# Substituição dos códigos das variáveis pelos nomes constantes nos datasets auxiliares para melhor entendimento do dataset (Tópico 2.2.5 do TCC)

# Unindo os datasets auxiliares com o df_importacao 
df_importacao = df_importacao.merge(df_ncm[['CO_NCM','NO_NCM_POR']],on='CO_NCM')
df_importacao = df_importacao.merge(df_pais[['CO_PAIS','NO_PAIS']],on='CO_PAIS')
df_importacao = df_importacao.merge(df_urf[['CO_URF','NO_URF']],on='CO_URF')
df_importacao = df_importacao.merge(df_via[['CO_VIA','NO_VIA']],on='CO_VIA')

# Criando a coluna NCM, concatenando o código e o nome da NCM 
df_importacao["NCM"] = df_importacao["CO_NCM"].map(str) + " - " + df_importacao["NO_NCM_POR"]

# Apagando as colunas dos códigos que foram utilizadas na união e o nome da NCM
df_importacao.drop(['CO_NCM','CO_PAIS','CO_URF','CO_VIA', 'NO_NCM_POR'],axis=1,inplace=True)

# Reordenando as colunas para melhor visualização
df_importacao = df_importacao[['CO_ANO','CO_MES','NCM','CO_UNID','NO_PAIS','SG_UF_NCM','NO_VIA','NO_URF','QT_ESTAT','KG_LIQUIDO','VL_FOB','VL_SEGURO','VL_FRETE']]

In [ ]:
# Visualização do cabeçalho do dataset df_importacao após as substituições (Tópico 2.2.5 do TCC)
df_importacao.head()

### Dolar Série Histórica
---
Link: [Dolar Serie Historica](https://www.cepea.esalq.usp.br/br/serie-de-preco/dolar.aspx)

In [ ]:
# Coletando os dados históricos das cotaões do Dolar Americano e carregando no dataset df_dolar (Tópico 2 do TCC)


#df_dolar = pd.read_csv("/content/drive/Shareddrives/TCC Daniel dados/export_series_dolar.csv",sep=';')


df_dolar = pd.read_csv("/content/export_series_dolar.csv",sep=';')

In [ ]:
# Informações do dataset df_urf (Tópico 2.3 do TCC)
df_dolar.info()

In [ ]:
# Tratando os dados do df_dolar para que possam expressar as cotações médias mensais do dólar a cada mês no período do escopo do trabalho (Tópico 3.3 do TCC)

# Criando as colunas ano, mes e COTACAO a partir da coluna do dataset df_dolar 
df_dolar["ano"] = df_dolar["Data da serie,DOLAR COMERCIAL ATUALIZADO AS 16H30"].apply(lambda x: str(x)[6:10])
df_dolar["mes"] = df_dolar["Data da serie,DOLAR COMERCIAL ATUALIZADO AS 16H30"].apply(lambda x: str(x)[3:5])
df_dolar["COTACAO"] = df_dolar["Data da serie,DOLAR COMERCIAL ATUALIZADO AS 16H30"].apply(lambda x: str(x)[11:])

# Apagando a coluna inicial do dataset df_dolar
df_dolar.drop(['Data da serie,DOLAR COMERCIAL ATUALIZADO AS 16H30'],axis=1,inplace=True)

# Substituindo a vírgula por ponto na string que representa a cotação e tirando as aspas
df_dolar["COTACAO"] = df_dolar["COTACAO"].str.replace(',','.').str.replace('"','')

# Alterando os tipos das colunas ano, mes e COTACAO
df_dolar["ano"] = df_dolar["ano"].astype(int)
df_dolar["mes"] = df_dolar["mes"].astype(int)
df_dolar["COTACAO"] = df_dolar["COTACAO"].astype(float)

# Filtrando dados dos últimos 10 anos (2012 - 2022)
df_dolar = df_dolar[df_dolar.ano >= 2012]

# Agrupando os valores por mês e ano utilizando a média das cotações 
df_dolar = df_dolar.groupby(['ano','mes']).mean()
df_dolar.reset_index(inplace=True)

# Informações do dataset df_dolar após as transformações
df_dolar.info()

## Verificação de dados nulos e registros duplicados dos datasets df_importacao e auxiliares

In [ ]:
# Verificação de missing values no df_importacao (Tópico 3.1 do TCC)
df_importacao.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_importacao (Tópico 3.1 do TCC)
df_importacao.duplicated().sum()

In [ ]:
# Verificação de missing values no df_pais (Tópico 3.2 do TCC)
df_pais.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_pais (Tópico 3.2 do TCC)
df_pais.duplicated().sum()

In [ ]:
# Verificação de missing values no df_ncm (Tópico 3.2 do TCC)
df_ncm.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_ncm (Tópico 3.2 do TCC)
df_ncm.duplicated().sum()

In [ ]:
# Verificação de missing values no df_via (Tópico 3.2 do TCC)
df_via.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_via (Tópico 3.2 do TCC)
df_via.duplicated().sum()

In [ ]:
# Verificação de missing values no df_urf (Tópico 3.2 do TCC)
df_urf.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_urf (Tópico 3.2 do TCC)
df_urf.duplicated().sum()

In [ ]:
# Verificação de missing values no df_dolar (Tópico 3.3 do TCC)
df_dolar.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_dolar (Tópico 3.3 do TCC)
df_dolar.duplicated().sum()

### Unindo Datasets - Dolar e Importações
---


In [ ]:
# Unindo os datasets df_importacao e df_dolar pelos valores do ano e do mês e carregando em um novo dataset df_total (Tópico 3.4 do TCC)

# Unindo os datasets df_importacao e df_dolar
df_total = df_importacao.merge(df_dolar,left_on=['CO_ANO','CO_MES'],right_on=['ano','mes'])

# Apagando as colunas repetidas
df_total.drop(['ano','mes'],axis=1,inplace=True)

# Reordenando as colunas para melhor visualização
df_total = df_total[['CO_ANO','CO_MES','NCM','CO_UNID','NO_PAIS','SG_UF_NCM','NO_VIA','NO_URF','QT_ESTAT','KG_LIQUIDO','COTACAO','VL_FOB','VL_SEGURO','VL_FRETE']]

# Informações de linhas e colunas do dataset df_total
df_total.info()

### Verificação de dados nulos e registros duplicados do dataset df_total

In [ ]:
# Verificação de missing values no df_total (Tópico 3.4 do TCC)
df_total.isnull().sum()

In [ ]:
# Verificação de registros duplicados no df_total (Tópico 3.4 do TCC)
df_total.duplicated().sum()

### Verificação de valores iguais a zero no dataset df_total

In [ ]:
# Verificando os registro que contém o valor 0 nas variáveis numéricas e mostrando a quantidade absoluta e percentual desses registros para cada variável (Tópico 3.5 do TCC)

# Verificando o total de linhas do dataset df_total
quant_total = len(df_total.index)

# Contando a quantidades de zeros de cada coluna e verificando o percentual correspondente em relação ao total de registros do df_total
df_contador = (df_total[['QT_ESTAT','KG_LIQUIDO','COTACAO','VL_FOB','VL_SEGURO','VL_FRETE']] == 0).sum()
df_contador = pd.DataFrame({'Coluna':df_contador.index, 'Quantidade':df_contador.values})
df_contador['porcentagem'] = df_contador.Quantidade.apply(lambda x: x/quant_total)
df_contador

In [ ]:
# Plotando a quantidade percentual de zeros de cada coluna do df_total em um gráfico (Tópico 3.5 do TCC)
fig, ax =plt.subplots(figsize=(6,7))
sns.barplot(x=df_contador.Coluna, y = df_contador.Quantidade)
plt.title("Quantidade de Zeros por coluna", fontsize=18)
for i,p in enumerate(ax.patches):
    ax.annotate('{:.2f}%'.format((df_contador.porcentagem[i])*100), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Substituindo os valroes 0 por 1 nas colunas QT_ESTAT, KG_LIQUIDO e VL_FOB no df_total (Tópico 3.5 do TCC)
df_total['QT_ESTAT'] = df_total[['QT_ESTAT']].replace(0,1)
df_total['KG_LIQUIDO'] = df_total[['KG_LIQUIDO']].replace(0,1)
df_total['VL_FOB'] = df_total[['VL_FOB']].replace(0,1)

## Análise e Exploração dos Dados
---


### Análise das NCM importadas
-----


In [ ]:
# Calculando os valores totais absolutos e percentuais das NCM pertencentes ao escopo no valor FOB total de importações (Tópico 4.1.1 do TCC)

# Agrupando o valor FOB total por NCM
df = df_total.groupby(['NCM']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]

# Criando coluna que indica o percentual de participação de cada NCM no valor FOB total
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

# Criando coluna extraindo somente o código da NCM com 8 dígitos
df["CO_NCM"] = df["NCM"].apply(lambda x: str(x)[:8])

In [ ]:
# Visualização gráfica dos percentuais de participação das NCM pertencentes ao escopo no valor FOB total de importações (Tópico 4.1.1 do TCC)

sns.set_theme(style="darkgrid")

# Inicializacao figura matplotlib
f, ax = plt.subplots(figsize=(15, 12))
bar = sns.barplot(x=list(df.VL_FOB)[:], y=list(df.CO_NCM)[:],
            label="Total", color="b")

  
for i,p in enumerate(ax.patches):
    ax.annotate('{:.1f}%'.format((df.porcentagem[i])*100), 
                   (p.get_width() + p.get_x(), p.get_y()), 
                   ha = 'center', va = 'center', 
                   xytext = (20, -12), 
                   textcoords = 'offset points')

    
ax.set_title("NCM mais importadas pelo Brasil, em FOB (U$)")
plt.show()

In [ ]:
# Criando dataset, a partir do df_total, com as colunas da NCM e do valor do frete (Tópico 4.1.1 do TCC)
df_dist_ncm = df_total[['NCM','VL_FRETE']]
df_dist_ncm["CO_NCM"] = df_dist_ncm["NCM"].apply(lambda x: str(x)[:8])

In [ ]:
# Plotando gráfico de distribuição stripplot do valor do frete por cada NCM (Tópico 4.1.1 do TCC)
fig, ax1 = plt.subplots(sharey=False, figsize=(20,10))
plt.title("Frete por NCM")
sns.stripplot(x='CO_NCM', y='VL_FRETE', data=df_dist_ncm, ax=ax1)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plotando gráfico de distribuição boxplot do valor do frete por cada NCM (Tópico 4.1.1 do TCC)
fig, ax2 = plt.subplots(sharey=False, figsize=(20,10))
plt.title("Frete por NCM")
sns.boxplot(x='CO_NCM', y='VL_FRETE', data=df_dist_ncm, showfliers = False, ax=ax2)
plt.xticks(rotation=45)
plt.show()
#plt.savefig('imagens/tipoCE2.png',bbox_inches='tight')

### Análise das Unidades Estatísticas

In [ ]:
# Criando dataset, a partir do df_total, com as colunas da NCM e código da unidade estatística (Tópico 4.1.2 do TCC)
df_unidest = df_total[['NCM','CO_UNID']]
df_unidest["NCM"] = df_unidest["NCM"].apply(lambda x: str(x)[:8])

In [ ]:
# Plotando gráfico de distribuição stripplot da unidade estatística por cada NCM (Tópico 4.1.2 do TCC)
fig, ax = plt.subplots(sharey=False, figsize=(20,10))
plt.title("Unidadde Estatística por NCM")
sns.stripplot(x='NCM', y='CO_UNID', data=df_unidest)
plt.xticks(rotation=30)
plt.show()

In [ ]:
# Contando a quantidade absoluta e percentual que cada unidade estatística aparece no df_total (Tópico 4.1.2 do TCC)
df_contador = df_unidest['CO_UNID'].value_counts()
df_contador = pd.DataFrame({'CO_UNID':df_contador.index, 'Quantidade':df_contador.values})
df_contador['porcentagem'] = df_contador.Quantidade.apply(lambda x: x/quant_total)*100
df_contador

In [ ]:
# Removendo as colunas CO_UNID e QT_ESTAT do df_total (Tópico 4.1.2 do TCC)
df_total.drop(['CO_UNID','QT_ESTAT'],axis=1,inplace=True)

### Análise dos Países Exportadores
-----


In [ ]:
# Verificando a quantidade de países distintos que aparecem no dataset df_total (Tópico 4.1.3 do TCC)
qt_paises = df_total[['NO_PAIS']].nunique()
qt_paises

In [ ]:
# Calculando os valores totais absolutos e percentuais dos países exportadores no valor FOB total de importações (Tópico 4.1.3 do TCC)

# Agrupando o valor FOB total por país exportador
df = df_total.groupby(['NO_PAIS']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]

# Criando coluna que indica o percentual de participação de cada país exportador no valor FOB total
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

In [ ]:
# Verificando o percentual que os países selecionados representam do valor FOB total (Tópico 4.1.3 do TCC)
fob_total = df.VL_FOB.sum()
maiores_paises = df.iloc[:37,:]
fob_maiores_paises = maiores_paises.VL_FOB.sum()
perc_paises = fob_maiores_paises/fob_total
print(f'O percentual que esses países representam no valor FOB é de {perc_paises:.1%}')

In [ ]:
# Substituindo os países que não pertencem ao conjunto de países que correspondem a 95% do valor FOB do total de importações pelo valor 'OUTROS' (Tópico 4.1.3 do TCC)

df_total.NO_PAIS = df_total.NO_PAIS.apply(lambda x : 'OUTROS' if x not in list(maiores_paises.NO_PAIS) else x)

In [ ]:
# Visualização gráfica dos percentuais de participação dos países exportadores no valor FOB total de importações (Tópico 4.1.3 do TCC)

df = df_total.groupby(['NO_PAIS']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

fig, ax =plt.subplots(figsize=(20,35))
sns.barplot(y=df['NO_PAIS'], x = df['VL_FOB'])
plt.title("País Exportador", fontsize=18)

for i,p in enumerate(ax.patches):
    ax.annotate('{:.1f}%'.format((df.porcentagem[i])*100), 
                   (p.get_width() + p.get_x(), p.get_y()), 
                   ha = 'center', va = 'center', 
                   xytext = (20, -15), 
                   textcoords = 'offset points')


plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plotando gráfico de distribuição stripplot do valor do frete por cada país exportador (Tópico 4.1.3 do TCC)
fig, ax1 = plt.subplots(sharey=False, figsize=(20,35))
plt.title("Frete por Unidade Estatística")
sns.stripplot(y='NO_PAIS', x='VL_FRETE', data=df_total, ax=ax1)
plt.show()

In [ ]:
# Plotando gráfico de distribuição boxplot do valor do frete por cada país exportador (Tópico 4.1.3 do TCC)
fig, ax2 = plt.subplots(sharey=False, figsize=(20,35))
plt.title("Frete por Unidade Estatística")
sns.boxplot(y='NO_PAIS', x='VL_FRETE', data=df_total, showfliers = False, ax=ax2)
plt.show()

### Análise dos Estados de Destino das Mercadorias
-----


In [ ]:
# Verificando a quantidade de estados de destino distintos que aparecem no dataset df_total (Tópico 4.1.4 do TCC)
qt_uf = df_total[['SG_UF_NCM']].nunique()
qt_uf

In [ ]:
# Calculando os valores totais absolutos e percentuais dos estados de destino no valor FOB total de importações (Tópico 4.1.4 do TCC)

# Agrupando o valor FOB total por estado de destino
df = df_total.groupby(['SG_UF_NCM']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]

# Criando coluna que indica o percentual de participação de cada estado de destino no valor FOB total
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)


In [ ]:
# Verificando o percentual que os estados de destino selecionados representam do valor FOB total (Tópico 4.1.4 do TCC)
fob_total = df.VL_FOB.sum()
maiores_destinos = df.iloc[:15,:]
fob_maiores_destinos = maiores_destinos.VL_FOB.sum()
perc_destinos = fob_maiores_destinos/fob_total
print(f'O percentual que esses destinos representam no valor FOB é de {perc_destinos:.1%}')

In [ ]:
# Substituindo os estados de destino que não pertencem ao conjunto de estados de destino que correspondem a 95% do valor FOB do total de importações pelo valor 'OUTROS' (Tópico 4.1.4 do TCC)
df_total.SG_UF_NCM = df_total.SG_UF_NCM.apply(lambda x : 'OUTROS' if x not in list(maiores_destinos.SG_UF_NCM) else x)

In [ ]:
# Visualização gráfica dos percentuais de participação dos estados de destino no valor FOB total de importações (Tópico 4.1.4 do TCC)
df = df_total.groupby(['SG_UF_NCM']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

fig, ax =plt.subplots(figsize=(20,6))
sns.barplot(x=df['SG_UF_NCM'], y = df['VL_FOB'])
plt.title("Estado de Destino", fontsize=18)
for i,p in enumerate(ax.patches):
   ax.annotate('{:.1f}%'.format((df.porcentagem[i])*100), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                  ha = 'center', va = 'center', 
                  xytext = (0, 9), 
                   textcoords = 'offset points')
plt.show()

In [ ]:
# Plotando gráfico de distribuição stripplot do valor do frete por cada estado de destino (Tópico 4.1.4 do TCC)
fig, ax1 = plt.subplots(sharey=False, figsize=(20,10))
plt.title("Frete por Estado de Destino")
sns.stripplot(x='SG_UF_NCM', y='VL_FRETE', data=df_total, ax=ax1)
plt.show()


In [ ]:
# Plotando gráfico de distribuição boxplot do valor do frete por cada estado de destino (Tópico 4.1.4 do TCC)
fig, ax2 = plt.subplots(sharey=False, figsize=(20,10))
plt.title("Frete por Estado de Destino")
sns.boxplot(x='SG_UF_NCM', y='VL_FRETE', data=df_total, showfliers = False, ax=ax2)
plt.show()

### Análise da Via de Transporte
-----


In [ ]:
# Verificando a quantidade de vias de transporte distintas que aparecem no dataset df_total (Tópico 4.1.5 do TCC)
qt_via = df_total[['NO_VIA']].nunique()
qt_via

In [ ]:
# Calculando os valores totais absolutos e percentuais das vias de transporte no valor FOB total de importações (Tópico 4.1.5 do TCC)
df = df_total.groupby(['NO_VIA']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)


In [ ]:
# Verificando o percentual que as vias de transporte selecionadas representam do valor FOB total (Tópico 4.1.5 do TCC)
fob_total = df.VL_FOB.sum()
maiores_vias = df.iloc[:3,:]
fob_maiores_vias = maiores_vias.VL_FOB.sum()
perc_vias = fob_maiores_vias/fob_total
print(f'O percentual que essas vias de transporte representam no valor FOB é de {perc_vias:.1%}')

In [ ]:
# Substituindo as vias que não pertencem ao conjunto de vias de transporte que correspondem a 95% do valor FOB do total de importações pelo valor 'OUTROS' (Tópico 4.1.5 do TCC)
df_total.NO_VIA = df_total.NO_VIA.apply(lambda x : 'OUTROS' if x not in list(maiores_vias.NO_VIA) else x)

In [ ]:
# Visualização gráfica dos percentuais de participação das vias de trasnporte no valor FOB total de importações (Tópico 4.1.5 do TCC)
df = df_total.groupby(['NO_VIA']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

fig, ax =plt.subplots(figsize=(10,8))
sns.barplot(x=df['NO_VIA'], y = df['VL_FOB'])
plt.title("Via de Transporte", fontsize=18)
for i,p in enumerate(ax.patches):
    ax.annotate('{:.1f}%'.format((df.porcentagem[i])*100), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plotando gráfico de distribuição stripplot do valor do frete por cada via de transporte (Tópico 4.1.5 do TCC)
fig, ax1 = plt.subplots(sharey=False, figsize=(10,8))
plt.title("Frete por Via de Transporte")
sns.stripplot(x='NO_VIA', y='VL_FRETE', data=df_total, ax=ax1)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Plotando gráfico de distribuição boxplot do valor do frete por cada via de transporte (Tópico 4.1.5 do TCC)
fig, ax2 = plt.subplots(sharey=False, figsize=(10,8))
plt.title("Frete por Via de Transporte")
sns.boxplot(x='NO_VIA', y='VL_FRETE', data=df_total, showfliers = False, ax=ax2)
plt.xticks(rotation=45)
plt.show()

### Análise da URF de Desembaraço


In [ ]:
# Verificando a quantidade de unidades de desembaraço distintas que aparecem no dataset df_total (Tópico 4.1.6 do TCC)
qt_urf = df_total[['NO_URF']].nunique()
qt_urf

In [ ]:
# Calculando os valores totais absolutos e percentuais das unidades de desembaraço no valor FOB total de importações (Tópico 4.1.6 do TCC)
df = df_total.groupby(['NO_URF']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

In [ ]:
# Verificando o percentual que as unidades de desembaraço selecionadas representam do valor FOB total (Tópico 4.1.6 do TCC)
fob_total = df.VL_FOB.sum()
maiores_urf = df.iloc[:30,:]
fob_maiores_urf = maiores_urf.VL_FOB.sum()
perc_urf = fob_maiores_urf/fob_total
print(f'O percentual que essas unidades representam no valor FOB é de {perc_urf:.1%}')

In [ ]:
# Substituindo as unidades que não pertencem ao conjunto de unidades de desembaraço que correspondem a 95% do valor FOB do total de importações pelo valor 'OUTROS' (Tópico 4.1.6 do TCC)
df_total.NO_URF = df_total.NO_URF.apply(lambda x : 'OUTROS' if x not in list(maiores_urf.NO_URF) else x)

In [ ]:
# Visualização gráfica dos percentuais de participação das unidades de desembaraço no valor FOB total de importações (Tópico 4.1.6 do TCC)
df = df_total.groupby(['NO_URF']).sum().sort_values(by=['VL_FOB'],ascending=False)
df = df[['VL_FOB']]
fob_total = df.VL_FOB.sum()
df['porcentagem'] = df.VL_FOB.apply(lambda x: x/fob_total)
df.reset_index(inplace=True)

fig, ax =plt.subplots(figsize=(20,30))
sns.barplot(y=df['NO_URF'], x = df['VL_FOB'])
plt.title("Unidade de Desembaraço", fontsize=18)
for i,p in enumerate(ax.patches):
    ax.annotate('{:.1f}%'.format((df.porcentagem[i])*100), 
                   (p.get_width() + p.get_x(), p.get_y()), 
                   ha = 'center', va = 'center', 
                   xytext = (25, -12), 
                   textcoords = 'offset points')

plt.show()

In [ ]:
# Plotando gráfico de distribuição stripplot do valor do frete por cada unidade de desembaraço (Tópico 4.1.6 do TCC)
fig, ax1 = plt.subplots(sharey=False, figsize=(20,30))
plt.title("Frete por URF de desembaraço")
sns.stripplot(y='NO_URF', x='VL_FRETE', data=df_total, ax=ax1)
plt.show()

In [ ]:
# Plotando gráfico de distribuição boxplot do valor do frete por cada unidade de desembaraço (Tópico 4.1.6 do TCC)
fig, ax2 = plt.subplots(sharey=False, figsize=(20,30))
plt.title("Frete por URF de desembaraço")
sns.boxplot(y='NO_URF', x='VL_FRETE', data=df_total, showfliers = False, ax=ax2)
plt.show()

### Análise Variáveis quantitativas


In [ ]:
# Plotagem do gráfico scatterplot para verificar a correlação entre as variáveis VL_FRETE e KG_LIQUIDO (Tópico 4.2 do TCC)
sns.scatterplot(x='VL_FRETE', y='KG_LIQUIDO', data=df_total)

In [ ]:
# Plotagem do gráfico scatterplot para verificar a correlação entre as variáveis VL_FRETE e COTACAO (Tópico 4.2 do TCC)
sns.scatterplot(x='VL_FRETE', y='COTACAO', data=df_total)

In [ ]:
# Plotagem do gráfico scatterplot para verificar a correlação entre as variáveis VL_FRETE e VL_FOB (Tópico 4.2 do TCC)
sns.scatterplot(x='VL_FRETE', y='VL_FOB', data=df_total)

In [ ]:
# Plotagem do gráfico scatterplot para verificar a correlação entre as variáveis VL_FRETE e VL_SEGURO (Tópico 4.2 do TCC)
sns.scatterplot(x='VL_FRETE', y='VL_SEGURO', data=df_total)

In [ ]:
# Plotagem do gráfico heatmap para verificar as correlações entre as variáveis quantitativas (Tópico 4.2 do TCC)
df_quantitativas = df_total[['KG_LIQUIDO','COTACAO','VL_FOB','VL_SEGURO','VL_FRETE']]

with sns.axes_style("white"):

    f, ax = plt.subplots(figsize=(15, 10))

    ax = sns.heatmap(df_quantitativas.corr(),annot=True)

In [ ]:
# Calculo dos valores totais, a cada ano, das variáveis 'KG_LIQUIDO', 'VL_FOB', 'VL_SEGURO' e 'VL_FRETE' excluindo o ano de 2022 por conter somente o mês de janeiro (Tópico 4.2 do TCC)
df_quantmensal = df_total[['CO_ANO','KG_LIQUIDO','VL_FOB','VL_SEGURO','VL_FRETE']]
df_quantmensal = df_quantmensal.groupby(['CO_ANO']).sum()
df_quantmensal.reset_index(inplace=True)
df_quantmensal = df_quantmensal[df_quantmensal.CO_ANO < 2022]
df_quantmensal.head()

In [ ]:
# Gráfico dos valores totais, a cada ano, das variáveis 'KG_LIQUIDO', 'VL_FOB', 'VL_SEGURO' e 'VL_FRETE' (Tópico 4.2 do TCC)
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Total VL_FRETE 2012 - 2021")
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.KG_LIQUIDO)
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_FOB)
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_SEGURO)
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_FRETE)
plt.legend(['KG_LIQUIDO','VL_FOB','VL_SEGURO','VL_FRETE'], fontsize=14)
plt.show()

In [ ]:
# Gráfico dos valores totais, a cada ano, da variável 'KG_LIQUIDO' (Tópico 4.2 do TCC)
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(15, 6))
ax.set_title("Total KG_LIQUIDO 2012 - 2021")
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.KG_LIQUIDO)
plt.show()

In [ ]:
# Gráfico dos valores totais, a cada ano, da variável 'VL_FOB' (Tópico 4.2 do TCC)
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(15, 6))
ax.set_title("Total VL_FOB 2012 - 2021")
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_FOB)
plt.show()

In [ ]:
# Gráfico dos valores totais, a cada ano, da variável 'VL_SEGURO' (Tópico 4.2 do TCC)
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(15, 6))
ax.set_title("Total VL_SEGURO 2012 - 2021")
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_SEGURO)
plt.show()

In [ ]:
# Gráfico dos valores totais, a cada ano, da variável 'VL_FRETE' (Tópico 4.2 do TCC)
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(15, 6))
ax.set_title("Total VL_FRETE 2012 - 2021")
plt.plot(df_quantmensal.CO_ANO,df_quantmensal.VL_FRETE)
plt.show()

In [ ]:
# Gráfico de distribuição boxplot, a cada ano, da variável 'COTACAO' (Tópico 4.2 do TCC)

# Configurando tema
sns.set_theme(style="darkgrid")

# Configurando tamanho da figura
fig, ax = plt.subplots(figsize=(20,12))
ax.set_title("Cotação Dolar 2012 - 2022")
sns.boxplot(x='CO_ANO', y='COTACAO', data=df_total, showfliers = False)
ax.set_ylabel("Cotação")
ax.set_xlabel("Anos")
plt.grid(True)
plt.show()

## Criação dos Modelos de Machine Learning e Análise dos Resultados
-----


### Preparando o dataset df_final

In [ ]:
# Criando uma cópia do dataset df_total, chamada de df_final. Nesse novo dataset, os dados serão transformados em numéricos para que possam ser aplicados os modelos de Machine Learning (Tópico 5 do TCC)
df_final = df_total.copy()

In [ ]:
# Tranformando as variáveis NCM e NO_URF em numéricas (Tópico 5 do TCC)
df_final["NCM"] = df_final["NCM"].apply(lambda x: str(x)[:8])
df_final["NO_URF"] = df_final["NO_URF"].apply(lambda x: str(x)[:7])
df_final["NO_URF"] = df_final["NO_URF"].replace('OUTROS',0)

In [ ]:
# Criação do dataset auxiliar df_sigla (Tópico 5 do TCC)
df_sigla = df_final[['SG_UF_NCM']]
df_sigla = df_sigla.drop_duplicates()
df_sigla.reset_index(inplace=True)
df_sigla.drop(['index'],axis=1,inplace=True)
df_sigla.reset_index(inplace=True)
df_sigla.at[8,'index']=0
df_sigla.at[0,'index']=8
df_sigla = df_sigla.sort_values(by='index')
df_sigla = df_sigla.rename(columns={"index":"CO_UF_NCM"})

In [ ]:
# Substituição dos nomes pelos códigos nas variáveis referentes ao país de exportação, ao estado de destino e à via de transporte. Para a substituição foram utilizados os datasets auxiliares (Tópico 5 do TCC)
df_final = df_final.merge(df_pais[['NO_PAIS','CO_PAIS']],on='NO_PAIS', how = 'left')
df_final = df_final.merge(df_sigla[['SG_UF_NCM','CO_UF_NCM']],on='SG_UF_NCM',how = 'left')
df_final = df_final.merge(df_via[['NO_VIA','CO_VIA']],on='NO_VIA',how = 'left')
df_final.fillna(value = 0,  
          inplace = True)
df_final = df_final[['CO_ANO','CO_MES','NCM','CO_PAIS','CO_UF_NCM','CO_VIA','NO_URF','KG_LIQUIDO','VL_FOB','COTACAO','VL_SEGURO','VL_FRETE']]
df_final

### Separando dados para treino e teste

In [ ]:
# Separando os dados de treino e de teste do dataset df_final, onde a variável é a VL_FRETE e as preditoras são todas as outras variáveis (Tópico 5 do TCC)
from sklearn.model_selection import train_test_split

X, y = df_final.drop(['VL_FRETE'],axis=1), df_final.VL_FRETE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Gráfico que representa a quantidade de registros nos dados de treino e de teste (Tópico 5 do TCC)

# Configurando Tema
sns.set_theme(style="darkgrid")

# Configurando tamanho da figura
fig, ax = plt.subplots(figsize=(5,10))
bar = sns.barplot(y=[X_train.shape[0],X_test.shape[0]],x=['Treino','Teste'])

for p in bar.patches:
    bar.annotate(int(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
    
ax.set_title("Divisão do Dataset para Treino e Teste")
plt.show()

## Gráfico de apresentação dos resultados

In [ ]:
# Função que plota um gráfico com a progessão aritmética dos valores do conjunto de teste e os valores preditos pelo modelo (Tópico 6 do TCC) 

def grafico_metricas(clf,X_train, X_test, y_train, y_test):
 
    soma = 0
    regr_y_test = []
    for val in y_test:
        soma += val
        regr_y_test.append(soma)

    soma = 0
    regr_y_predict = []
    for val in clf.predict(X_test):
        soma += val
        regr_y_predict.append(soma)

    # Configurando Tema
    sns.set_theme(style="darkgrid")

    # Configurando grafico
    fig, ax = plt.subplots(figsize=(12,12))

    plt.plot(range(len(regr_y_test)),regr_y_test,label='Gabarito')

    plt.plot(range(len(regr_y_predict)),regr_y_predict,label='Predição')

    ax.set_title('Progressão Aritmética Predição e Gabarito')

    plt.legend()
    plt.show()

    return

## Decision Tree

### Rodando o modelo com os parâmetros padrões (Tópico 5.1 do TCC)

In [ ]:
# Imports necessários ao modelo
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
tree = DecisionTreeRegressor(random_state=42)

In [ ]:
tree.fit(X_train,y_train)

### Resultados do modelo utilizando os parâmetros padrões (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos (Tópico 6 do TCC)
predict = tree.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo Decision Tree com parâmetros padrões (Tópico 6 do TCC) 
grafico_metricas(tree,X_train, X_test, y_train, y_test)

### Utilizando a biblioteca GridSearchCV para buscar os melhores parâmetros para o modelo (Tópico 5.1 do TCC)

In [ ]:
tree_hiper = DecisionTreeRegressor(random_state=42)

In [ ]:
parametros_tree_hiper = {'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
                   'min_samples_split':[2,3,4,5,6,7,8,9,10],}

In [ ]:
grid_tree = GridSearchCV(estimator=tree_hiper, param_grid=parametros_tree_hiper,cv=10,n_jobs=-1)

In [ ]:
grid_tree.fit(X_train,y_train)

In [ ]:
grid_tree.best_params_

### Resultados do modelo utilizando os parâmetros otimizados (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos, utilizando os melhores parâmetros para esse modelo (Tópico 6 do TCC)
tree_final = DecisionTreeRegressor(random_state=42,max_depth=15,min_samples_split=10)
tree_final.fit(X_train,y_train)

predict = tree_final.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo Decision Tree com parâmetros otimizados (Tópico 6 do TCC) 
grafico_metricas(tree_final,X_train, X_test, y_train, y_test)

## K-Neighbors

### Rodando o modelo com os parâmetros padrões (Tópico 5.2 do TCC)

In [ ]:
# Imports necessários ao modelo
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsRegressor()

In [ ]:
knn.fit(X_train,y_train)

### Resultados do modelo utilizando os parâmetros padrões (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos (Tópico 6 do TCC)
predict = knn.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo K-Neighbors com parâmetros padrões (Tópico 6 do TCC) 
grafico_metricas(knn,X_train, X_test, y_train, y_test)

### Utilizando a biblioteca GridSearchCV para buscar os melhores parâmetros para o modelo (Tópico 5.2 do TCC)

In [ ]:
knn_hiper = KNeighborsRegressor()

In [ ]:
parametros_knn_hiper = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
                   'weights':['uniform','distance'],
                   'metric':['euclidean','manhattan']}

In [ ]:
grid_knn = GridSearchCV(knn_hiper, parametros_knn_hiper,verbose=1,cv=10,n_jobs=-1)

In [ ]:
grid_knn.fit(X_train,y_train)

In [ ]:
grid_knn.best_params_

### Resultados do modelo utilizando os parâmetros otimizados (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos, utilizando os melhores parâmetros para esse modelo (Tópico 6 do TCC)
knn_final = KNeighborsRegressor(metric='manhattan',n_neighbors=15, weights='distance')
knn_final.fit(X_train,y_train)

predict = knn_final.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo K-Neighbors com parâmetros otimizados (Tópico 6 do TCC) 
grafico_metricas(knn_final,X_train, X_test, y_train, y_test)

## Random Forest

### Rodando o modelo com os parâmetros padrões (Tópico 5.3 do TCC)

In [ ]:
# Imports necessários ao modelo
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
rfr = RandomForestRegressor(random_state=42)

In [ ]:
rfr.fit(X_train,y_train)

### Resultados do modelo utilizando os parâmetros padrões (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos (Tópico 6 do TCC)
predict = rfr.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo Random Forest com parâmetros padrões (Tópico 6 do TCC) 
grafico_metricas(rfr,X_train, X_test, y_train, y_test)

### Utilizando a biblioteca GridSearchCV para buscar os melhores parâmetros para o modelo (Tópico 5.3 do TCC)

In [ ]:
rfr_hiper = RandomForestRegressor(random_state=42)

In [ ]:
parametros_rfr_hiper = {
                    'n_estimators':[100,200],
                   'max_features':[None,'auto','log2'],
                   'max_depth':[None,1,2,3,4],
                   'min_samples_split':[2,3,4,5,6],                   
                   'min_samples_leaf':[1,2,3,4],
                   'bootstrap':[True,False]
                      }

In [ ]:
grid_rfr = GridSearchCV(rfr_hiper, parametros_rfr_hiper,cv=3,n_jobs=-1)

In [ ]:
grid_rfr.fit(X_train,y_train)

In [ ]:
grid_rfr.best_params_

### Resultados do modelo utilizando os parâmetros otimizados (Tópico 6 do TCC)

In [ ]:
# Cálculo das métricas R2 e MAE, utilizadas para a comparação dos modelos, utilizando os melhores parâmetros para esse modelo (Tópico 6 do TCC)
rfr_final = RandomForestRegressor(random_state=42,bootstrap=False,max_depth=None,max_features='log2',min_samples_leaf=1,min_samples_split=2,n_estimators=100)
rfr_final.fit(X_train,y_train)

predict = rfr_final.predict(X_test)
r2 = r2_score(y_test, predict)
mae = median_absolute_error(y_test, predict)
print(f'\nR2: {r2}\nMAE: {mae}\n')

In [ ]:
# Gráfico com a progessão aritmética dos valores do conjunto de teste (gabarito) e os valores preditos pelo modelo Random Forest com parâmetros otimizados (Tópico 6 do TCC) 
grafico_metricas(rfr_final,X_train, X_test, y_train, y_test)